In [1]:
import requests, json
import pandas as pd

In [6]:
API_KEY = '본인의 api키'

In [7]:
df = pd.read_csv('C:/Users/gherwt/KDT30/p1/seongdong.csv', encoding = "cp949")
df['address'] = '서울특별시 성동구 ' + df['법정동'] + ' ' + df['지번']
df['address'] 

0        서울특별시 성동구 하왕십리동 890-401
1        서울특별시 성동구 하왕십리동 890-114
2              서울특별시 성동구 홍익동 125
3              서울특별시 성동구 홍익동 367
4              서울특별시 성동구 도선동 188
                  ...           
12206        서울특별시 성동구 용답동 45194
12207        서울특별시 성동구 용답동 52-19
12208        서울특별시 성동구 용답동 52-19
12209        서울특별시 성동구 용답동 52-19
12210        서울특별시 성동구 용답동 26-20
Name: address, Length: 12211, dtype: object

In [ ]:
# 주소를 좌표로 변환
def convert_address_to_coordinates(address):
    api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}
    params = {"query": address}
    
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            coordinates = result["documents"][0]["y"], result["documents"][0]["x"]
            return coordinates
        else:
            
            
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [9]:
# 데이터프레임에 좌표 추가
def add_coordinates_to_dataframe(df, address_column):
    latitudes = []
    longitudes = []

    for address in df[address_column]:
        coordinates = get_location(address)
        if coordinates:
            latitudes.append(coordinates[0])
            longitudes.append(coordinates[1])
        else:
            latitudes.append(None)
            longitudes.append(None)

    df["Lat"] = latitudes
    df["Long"] = longitudes

In [10]:
get_location(df, 'address')
df.head()

,Unnamed: 0,지역코드,법정동,지번,연립다세대,건축년도,층,전용면적,년,월,...,보증금액,월세금액,계약구분,계약기간,갱신요구권사용,종전계약보증금,종전계약월세,address,Lat,Long
0,0,11200,하왕십리동,890-401,광진연립B동,1986,3.0,39.70,2015,1,...,15000,0,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 하왕십리동 890-401,37.5636035234975,127.028500837057
1,1,11200,하왕십리동,890-114,수빌,2012,4.0,13.97,2015,1,...,3000,30,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 하왕십리동 890-114,37.5632879114037,127.028095966494
2,2,11200,홍익동,125,동아맨션,1977,3.0,85.36,2015,1,...,17000,0,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 홍익동 125,37.5658216064465,127.03604346832
3,3,11200,홍익동,367,리더스파크,2014,2.0,27.45,2015,1,...,19000,0,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 홍익동 367,37.5659973373081,127.031506133012
4,4,11200,도선동,188,도선빌,2012,5.0,17.56,2015,1,...,6000,30,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 도선동 188,37.5645597759861,127.033229707255


In [41]:
seongdong_lat_long = pd.DataFrame({'dong': df['법정동'],'address': df['address'], 'jibun': df['지번'], 'lat': df["Lat"], 'long': df["Long"]})

In [42]:
seongdong_lat_long.to_csv('seongdong_lat_long.csv', index = False)